# Qcodes example with Attocube ANC300

## Imports

In [26]:
import qcodes as qc
from qcodes.instrument_drivers.Attocube.Attocube_ANC300 import ANC300

import time

#ef test_ANC300_param(ANC300, param):
#   print("Testing parameter: {!s}".format(param.name))
#   mode = ANC300
#   print("Current mode: ")
#   val_0 = qcodes_param.get()
#   print("Initial value of {!s} = {} {}".format(qcodes_param.name, val_0 , qcodes_param.unit))
#   print("Valid values for {!s}:\n{!r}".format(qcodes_param.name, qcodes_param.vals.valid_values))
#   for val in qcodes_param.vals.valid_values:
#       print("Set {!s} = {} {}".format(qcodes_param.name, val, qcodes_param.unit))
#       qcodes_param.set(val)
#       print("Got {!s} = {} {}".format(qcodes_param.name, qcodes_param.get(), qcodes_param.unit))
#   qcodes_param.set(val_0)
#   print()

def test_qcodes_param(qcodes_param):
    val_0 = qcodes_param.get()
    print("Initial value of {!s} = {} {}".format(qcodes_param.name, val_0 , qcodes_param.unit))
    print("Valid values for {!s}:\n{!r}".format(qcodes_param.name, qcodes_param.vals.valid_values))
    for val in qcodes_param.vals.valid_values:
        print("Set {!s} = {} {}".format(qcodes_param.name, val, qcodes_param.unit))
        try:
            qcodes_param.set(val)
        except Exception as e:
            print(e)
        print("Got {!s} = {} {}".format(qcodes_param.name, qcodes_param.get(), qcodes_param.unit))
    qcodes_param.set(val_0)
    print()

def print_qcodes_param(qcodes_param):
    print("Got {!s} = {} {}\n".format(qcodes_param.name, qcodes_param.get(), qcodes_param.unit))

## Connect instrument

In [2]:
piezo_ctrl = ANC300("attocube_piezo_controller","tcpip0::192.168.1.2::7230::socket")

## Get and print instrument and submodules parameters

In [31]:
piezo_ctrl.print_readable_snapshot(update=True)

attocube_piezo_controller:
	parameter      value
--------------------------------------------------------------------------------
IDN             :	{'vendor': 'attocube', 'model': 'ANC300', 'serial': 'ANC300...
instrument_info :	attocube ANC300 controller version 1.0.0-1271 2012-04-18 16...
serial_no       :	ANC300B-C-1011-1004010 
timeout         :	5 (s)
attocube_piezo_controller_axis1:
	parameter        value
--------------------------------------------------------------------------------
ac_in             :	False 
capacitance       :	0.6823 (nF)
dc_in             :	False 
mode              :	gnd 
offset_voltage    :	0 (V)
outp_filter       :	off (Hz)
output_voltage    :	0 (V)
serial_no         :	ANM300B-M-0945-1003041 
step_amplitude    :	40 (V)
step_down_pattern :	[255, 254, 253, 252, 251, 250, 249, 248, 247, 246, 245, 2...
step_frequency    :	500 (Hz)
step_up_pattern   :	[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16...
trigger_down_pin  :	2 
trigger_up_pin    :	6 
atto

## Test setting parameters and calling functions

In [35]:
print(piezo_ctrl.axis1.mode.set('gnd'))
print(piezo_ctrl.axis1.step_up_cont())
time.sleep(2)
piezo_ctrl.axis1.stop()

None


ANC300GenericError: ('Axis in wrong mode', "writing 'stepu 1 c' to <ANC300: attocube_piezo_controller>")

In [4]:
axis = piezo_ctrl.axis1

In [24]:
axis.mode.vals.valid_values

['gnd', 'inp', 'cap', 'stp', 'off', 'stp+', 'stp-']

In [30]:
test_qcodes_param(axis.mode)

"""
Params/funcs that change mode (only for some values ?):
    - step_amplitude (only when set to 0 ?)
    - outp_filter 
    - offset_voltage
    - ac_in, dc_in
    - 
"""

print("# Capacitance meas testing\n")

axis.mode.set('cap')
axis.wait_capacitance_updated()
print_qcodes_param(axis.capacitance)

axis.update_capacitance()
axis.wait_capacitance_updated()
print_qcodes_param(axis.capacitance)

print("# General testing\n")
for mode in axis.mode.vals.valid_values:
    print("## Mode: {}\n".format(mode))
    try:
        axis.mode.set(mode)
        test_qcodes_param(axis.step_frequency)
        test_qcodes_param(axis.step_amplitude) #  requires more tests (changes mode when set)
        print_qcodes_param(axis.mode)
        axis.mode.set(mode)
        test_qcodes_param(axis.ac_in) #  requires more tests (changes mode when set)
        print_qcodes_param(axis.mode)
        axis.mode.set(mode)
        test_qcodes_param(axis.dc_in) #  requires more tests (changes mode when set)
        print_qcodes_param(axis.mode)
        axis.mode.set(mode)
        test_qcodes_param(axis.offset_voltage) #  requires more tests (changes mode when set)
        print_qcodes_param(axis.mode)
        axis.mode.set(mode)
        test_qcodes_param(axis.outp_filter) #  requires more tests (changes mode when set)
        print_qcodes_param(axis.mode)
        axis.mode.set(mode)
        print_qcodes_param(axis.output_voltage)
        print_qcodes_param(axis.capacitance) # weird values
        print_qcodes_param(axis.mode)
        axis.step_up_single()
        axis.wait_steps_end()
        axis.step_up(1000)
        axis.wait_steps_end()
        axis.step_up_cont()
        time.sleep(0.5)
        axis.stop()
        axis.step_down_single()
        axis.wait_steps_end()
        axis.step_down(1000)
        axis.wait_steps_end()
        axis.step_down_cont()
        time.sleep(0.5)
        axis.stop()
    except Exception as e:
        print(e)
    finally:
        print()

print()

#test_qcodes_param(axis.trigger_up_pin) # valid values not well defined enough yet
#test_qcodes_param(axis.trigger_down_pin)

#test_qcodes_param(axis.step_up_pattern) #  valid_values not well defined yet
#test_qcodes_param(axis.step_down_pattern) #  valid_values not well defined yet

#axis.mode.set('stp+')
#axis.mode.set('stp-')
#axis.mode.set('inp')
#axis.mode.set('off')
axis.mode.set('gnd')

Initial value of mode = gnd 
Valid values for mode:
['gnd', 'inp', 'cap', 'stp', 'off', 'stp+', 'stp-']
Set mode = gnd 
Got mode = gnd 
Set mode = inp 
Got mode = inp 
Set mode = cap 
Got mode = cap 
Set mode = stp 
Got mode = stp 
Set mode = off 
Got mode = off 
Set mode = stp+ 
Got mode = stp+ 
Set mode = stp- 
Got mode = stp- 

Got capacitance = None nF

Got capacitance = None nF

# General testing


## Mode: gnd

Initial value of step_frequency = 500 Hz
Valid values for step_frequency:
[1, 10000]
Set step_frequency = 1 Hz
Got step_frequency = 1 Hz
Set step_frequency = 10000 Hz
Got step_frequency = 500 Hz

Initial value of step_amplitude = 40.0 V
Valid values for step_amplitude:
[0, 150]
Set step_amplitude = 0 V
Got step_amplitude = 0.0 V
Set step_amplitude = 150 V
Got step_amplitude = 40.0 V

Got mode = gnd 

Initial value of ac_in = False 
Valid values for ac_in:
[True, False]
Set ac_in = True 
Got ac_in = True 
Set ac_in = False 
Got ac_in = False 

Got mode = gnd 

Initial value


## Mode: stp-

Initial value of step_frequency = 500 Hz
Valid values for step_frequency:
[1, 10000]
Set step_frequency = 1 Hz
Got step_frequency = 1 Hz
Set step_frequency = 10000 Hz
Got step_frequency = 500 Hz

Initial value of step_amplitude = 40.0 V
Valid values for step_amplitude:
[0, 150]
Set step_amplitude = 0 V
Got step_amplitude = 0.0 V
Set step_amplitude = 150 V
Got step_amplitude = 40.0 V

Got mode = stp- 

Initial value of ac_in = False 
Valid values for ac_in:
[True, False]
Set ac_in = True 
Got ac_in = True 
Set ac_in = False 
Got ac_in = False 

Got mode = stp- 

Initial value of dc_in = False 
Valid values for dc_in:
[True, False]
Set dc_in = True 
Got dc_in = True 
Set dc_in = False 
Got dc_in = False 

Got mode = stp- 

Initial value of offset_voltage = 0.0 V
Valid values for offset_voltage:
[0, 150]
Set offset_voltage = 0 V
Got offset_voltage = 0.0 V
Set offset_voltage = 150 V
Got offset_voltage = 40.0 V

Got mode = stp- 

Initial value of outp_filter = off Hz
Valid v

## Close instrument

In [85]:
piezo_ctrl.close()